ПРИМЕР ИСПОЛЬЗОВАНИЯ ORM

In [1]:
from sqlalchemy import Column, Integer, String


class User(Base):
    id = Column(Integer, primary_key=True)
    name = Column(String)

Атрибуты id & name не передаются через __new__ и не привязываются к инстансу в __init__

In [ ]:
PEP 487
#?3.6+
#упрощенная настройка создания классов
#МЕНЯЕТ способ использования дескрипторов 

In [3]:
# нехватает валидации данных
class Order:
    def __init__(self, name, price, quantity):
        self.name = name
        self.price = price
        self.quantity = quantity
    
    def total(self):
        return self.price * self.quantity


apple_order = Order('apple', 1, 10)
apple_order.total()

10

In [ ]:
ВМЕСТО СОЗДАНИЯ API НА ГЕТТЕРАХ/СЕТТЕРАХ ИСПОЛЬЗУЕМ PROPERTY

In [ ]:
преобразуем attr в non-negative св-во

In [10]:
class Order:
    def __init__(self, name, price, quantity):
        self.name = name
        self.price = price
        # избегаем recursion error(хз что за recursion, но при __init__
        # метод будет перезаписан аргументом)
        self._quantity = quantity
    
    # теперь это attr
    @property
    def quantity(self):
        return self._quantity
    
    @quantity.setter
    def quantity(self, value):
        if value < 0:
            raise ValueError('Cannot be negative.')
        self._quantity = value

apple_order = Order('apple', 1, 10)

apple_order = Order('apple', 1, 10)
print(apple_order.quantity)
apple_order.quantity = 20
print(apple_order.quantity)

10
20


In [11]:
apple_order.quantity = -1

ValueError: Cannot be negative.

In [14]:
wrong_order = Order('pineapples', 1, -10)

In [15]:
wrong_order.quantity

-10

In [16]:
wrong_order.quantity = - 20

ValueError: Cannot be negative.

In [ ]:
class Order:
    def __init__(self, quantity):
        #вызывает setter в первый раз
        self.quantity = quantity
    
    @property
    def quantity(self):
        print('getter')
        return self.quantity
    
    @quantity.setter
    def quantity(self, value):
        print('setter')
        #вызовет сама себя
        self.quantity = value

apple_order = Order(10)
# apple_order.quantity
#Вызовет recursion error

setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter
setter

In [ ]:
class Order:
    # очень похоже на orm
    # создаем obj NonNegative, явно передаем имя attr для доступа
    # через __dict__
    # передача имени 'price' - избыточно
    price = NonNegative('price')
    quantity = NonNegative('quantity')
    
    def __init__(self, name, price, quantity):
        self._name = name
        self._price = price
        self._quantity = quantity
    
    def total(self):
        return self.price * self.quantity


class NonNegative:
    def __init__(self, name):
        self.name = name
    
    def __get__(self, instance, owner):
        # вместо использования getattr - напрямую обращаемся к __dict
        # чтобы избежать recursion error тк getattr по идее будет вызывать __get__
        # (тк обращение через встроенные fx) будут так-же перехватываться 
        # протоколом дескрипторов
        return instance.__dict__[self.name]
    
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Cannot be negative.')
        instanse.__dict__[self.name] = value

In [ ]:
class NonNegative:
    ...
    def __set_name__(self, owner, name):
        self.name = name

In [1]:
class NonNegative:
    def __get__(self, instance, owner):
        return instance.__dict__[self.name]
    
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Cannot be negative.')
        instance.__dict__[self.name] = value
    
    def __set_name__(self, owner, name):
        self.name = name
    

class Order:
    price = NonNegative()
    quantity = NonNegative()
    
    def __init__(self, name, price, quantity):
        self._name = name
        self.price = price
        self.quantity = quantity
    
    def total(self):
        return self.price * self.quantity

apple_order = Order('apple', 1, 10)
apple_order.total()

10

In [2]:
apple_order.price = -10

ValueError: Cannot be negative.

In [3]:
wrong_order = Order('wrong', 1, -1)

ValueError: Cannot be negative.